In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,OneHotEncoder
import pandas as pd
import tensorflow as tf

In [2]:
# import the data file
charity_df = pd.read_csv('charity_data.csv')
charity_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [3]:
# Drop the name name and the EIN
charity_df = charity_df.drop(["EIN", "NAME"], axis=1)

In [4]:
# Generate our categorical variable list
charity_cat = charity_df.dtypes[charity_df.dtypes == "object"].index.tolist()
charity_cat

['APPLICATION_TYPE',
 'AFFILIATION',
 'CLASSIFICATION',
 'USE_CASE',
 'ORGANIZATION',
 'INCOME_AMT',
 'SPECIAL_CONSIDERATIONS']

In [5]:
# Check the number of unique values in each column
charity_df[charity_cat].nunique()

APPLICATION_TYPE          17
AFFILIATION                6
CLASSIFICATION            71
USE_CASE                   5
ORGANIZATION               4
INCOME_AMT                 9
SPECIAL_CONSIDERATIONS     2
dtype: int64

In [7]:
# Bin classification
classfication_counts = charity_df.CLASSIFICATION.value_counts()

# Determine which values to replace
replace_classfication = list(classfication_counts[classfication_counts < 700].index)

# Replace in DataFrame
for classification in replace_classfication:
    charity_df.CLASSIFICATION = charity_df.CLASSIFICATION.replace(classification,"Other-classification")


# Check to make sure binning was successful
charity_df.CLASSIFICATION.value_counts()

C1000                   17326
C2000                    6074
C1200                    4837
C3000                    1918
C2100                    1883
Other-classification     1484
C7000                     777
Name: CLASSIFICATION, dtype: int64

In [8]:
application_type_counts = charity_df.APPLICATION_TYPE.value_counts()

In [9]:
# Determine which values to replace
replace_application_type = list(application_type_counts[application_type_counts < 700].index)
replace_application_type

['T10', 'T9', 'T13', 'T12', 'T2', 'T25', 'T14', 'T29', 'T15', 'T17']

In [10]:
# # Replace in DataFrame
for app_type in replace_application_type:
    charity_df.APPLICATION_TYPE = charity_df.APPLICATION_TYPE.replace(app_type,"Other")


# Check to make sure binning was successful
charity_df.APPLICATION_TYPE.value_counts()

T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
Other      804
T8         737
T7         725
Name: APPLICATION_TYPE, dtype: int64

In [11]:
income_amt_counts = charity_df.INCOME_AMT.value_counts()
income_amt_counts

0                24388
25000-99999       3747
100000-499999     3374
1M-5M              955
1-9999             728
10000-24999        543
10M-50M            240
5M-10M             185
50M+               139
Name: INCOME_AMT, dtype: int64

In [12]:
# make a bin for 5m+
# Determine which values to replace
replace_income_amt = list(income_amt_counts[income_amt_counts < 300].index)
replace_income_amt

['10M-50M', '5M-10M', '50M+']

In [13]:
# Replace in DataFrame
for income in replace_income_amt:
    charity_df.INCOME_AMT = charity_df.INCOME_AMT.replace(income,"5M+")

# Check to make sure binning was successful
charity_df.INCOME_AMT.value_counts()

0                24388
25000-99999       3747
100000-499999     3374
1M-5M              955
1-9999             728
5M+                564
10000-24999        543
Name: INCOME_AMT, dtype: int64

In [14]:
# Create a OneHotEncoder instance
enc = OneHotEncoder(sparse=False)

# Fit and transform the OneHotEncoder using the categorical variable list
encode_df = pd.DataFrame(enc.fit_transform(charity_df[charity_cat]))

# Add the encoded variable names to the DataFrame
encode_df.columns = enc.get_feature_names(charity_cat)
encode_df.head()

,APPLICATION_TYPE_Other,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,APPLICATION_TYPE_T7,APPLICATION_TYPE_T8,AFFILIATION_CompanySponsored,AFFILIATION_Family/Parent,...,ORGANIZATION_Trust,INCOME_AMT_0,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_5M+,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
4,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0


In [15]:
# Merge one-hot encoded features and drop the originals
charity_df = charity_df.merge(encode_df,left_index=True, right_index=True)
charity_df = charity_df.drop(charity_cat,1)
charity_df.head()

,STATUS,ASK_AMT,IS_SUCCESSFUL,APPLICATION_TYPE_Other,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,APPLICATION_TYPE_T7,...,ORGANIZATION_Trust,INCOME_AMT_0,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_5M+,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
0,1,5000,1,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,1,108590,1,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,1,5000,0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,1,6692,1,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
4,1,142590,1,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0


In [16]:
# Split our preprocessed data into our features and target arrays
y = charity_df["IS_SUCCESSFUL"].values
X = charity_df.drop(["IS_SUCCESSFUL"],1).values

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78)

In [17]:
# Create a StandardScaler instance
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)
X_test_scaled

array([[ 0.01394306, -0.03336766, -0.15558816, ..., -0.12694995,
         0.02789425, -0.02789425],
       [ 0.01394306,  0.02868029, -0.15558816, ...,  7.87712001,
         0.02789425, -0.02789425],
       [ 0.01394306, -0.03348439, -0.15558816, ..., -0.12694995,
         0.02789425, -0.02789425],
       ...,
       [ 0.01394306, -0.03348439, -0.15558816, ..., -0.12694995,
         0.02789425, -0.02789425],
       [ 0.01394306, -0.03348439, -0.15558816, ..., -0.12694995,
         0.02789425, -0.02789425],
       [ 0.01394306, -0.03348439, -0.15558816, ..., -0.12694995,
         0.02789425, -0.02789425]])

In [39]:
# Define the model - deep neural net
number_input_features = len(X_train[0])
nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(
    tf.keras.layers.Dense(units=10, input_dim=number_input_features, activation="relu")
)

nn.add(tf.keras.layers.Dense(units=4, activation="relu"))
# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_13 (Dense)             (None, 10)                420       
_________________________________________________________________
dense_14 (Dense)             (None, 4)                 44        
_________________________________________________________________
dense_15 (Dense)             (None, 1)                 5         
Total params: 469
Trainable params: 469
Non-trainable params: 0
_________________________________________________________________


In [40]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [ ]:
# Fit the model to the training data
fit_model = nn.fit(X_train_scaled, y_train, epochs=50)

Train on 25724 samples
Epoch 1/50
25724/25724 [==============================] - 3s 101us/sample - loss: 0.6355 - accuracy: 0.6571
Epoch 2/50
25724/25724 [==============================] - 2s 65us/sample - loss: 0.5695 - accuracy: 0.7244
Epoch 3/50
25724/25724 [==============================] - 2s 73us/sample - loss: 0.5610 - accuracy: 0.7287
Epoch 4/50
25724/25724 [==============================] - 2s 77us/sample - loss: 0.5553 - accuracy: 0.7303
Epoch 5/50
25724/25724 [==============================] - 2s 73us/sample - loss: 0.5535 - accuracy: 0.7313
Epoch 6/50
25724/25724 [==============================] - 2s 74us/sample - loss: 0.5525 - accuracy: 0.7302
Epoch 7/50
25724/25724 [==============================] - 2s 68us/sample - loss: 0.5516 - accuracy: 0.7310
Epoch 8/50
25724/25724 [==============================] - 2s 72us/sample - loss: 0.5508 - accuracy: 0.7306
Epoch 9/50
25724/25724 [==============================] - 2s 71us/sample - loss: 0.5509 - accuracy: 0.7319
Epoch 10/50
2

In [ ]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")